In [12]:
from tqdm import tqdm
import numpy as np
import re, collections
import pickle
from multiprocessing import Pool, Process
import multiprocessing
from functools import partial
from math import log

In [13]:
!pip install cython
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [18]:
a_file = open("/content/drive/MyDrive/74.pkl", "rb")
vocab = pickle.load(a_file)

In [ ]:
%%cython

def get_vocab(filename):
    
    vocab = {}
    
    for line in open(filename, 'r', encoding='utf-8'):

        line = line.strip()

        key = ' '.join(list(line))

        if key in vocab.keys():
            vocab[key] += 1
        else:
            vocab[key] = 1

    return vocab

def get_freq(vocab, max_len):

    freqs = {}
    symbol_freqs = {}
    all = 0
    all_2 = 0


    for line, freq in vocab.items():

        symbols = line.split()

        all += len(symbols)

        for symbol_i in range(len(symbols)-1):

            key = symbols[symbol_i]
            if key in symbol_freqs.keys():
                symbol_freqs[key] += freq
            else:
                symbol_freqs[key] = freq


            if max_len >= len(symbols[symbol_i] + symbols[symbol_i+1]):
                all_2 += freq
                
                key = (symbols[symbol_i], symbols[symbol_i+1])
                if key in freqs.keys():
                    freqs[key] += freq
                else:
                    freqs[key] = freq

            key = symbols[-1]
            if key in symbol_freqs.keys():
                symbol_freqs[key] += freq
            else:
                symbol_freqs[key] = freq

    for key in freqs.keys():
        freqs[key] = (freqs[key]/all_2) / ((symbol_freqs[key[0]]/all)  * (symbol_freqs[key[1]]/all))
    
    return freqs


def merge_vocab(pair, v_in):

    v_out = {}

    bigram = ''.join(pair)
    pair = pair[0] + ' ' + pair[1]

    for line in v_in:

        w_out = line.replace(pair,bigram)
        v_out[w_out] = v_in[line]

    return v_out
    
def get_tokens(vocab):

    tokens = set({})

    for line in vocab.keys():
        line = line.split()

        for token in line:
            tokens.add(token)
    
    return tokens

In [ ]:
vocab = get_vocab('/content/sentences_for_task1.txt')

max_len = 16
num_merges = 15000

t = tqdm(np.arange(num_merges))
for i in t:

    pairs = get_freq(vocab, max_len)
    if len(pairs) == 0: break
    if max(pairs.values()) < 2: break
    
    pair = max(pairs, key=pairs.get)
    

    vocab = merge_vocab(pair, vocab)

    if i % 1000 == 0:
        tokens = get_tokens(vocab)
        mt = np.mean([len(token) for token in tokens])

        t.set_postfix({'mean_token_len': mt, 'vocab_size': len(tokens)})

In [19]:
 tokens = get_tokens(vocab)

In [20]:
np.mean([len(token) for token in tokens])

12.901111280457501

In [22]:
score = 0

for i, line in enumerate(open('/content/test_for_task1.txt', 'r')):
    w1,w2 = line.split()

    if w1 in tokens and w2 not in tokens:
        score+=1
    elif w1 not in tokens and w2 in tokens:
        score+=0
    else:
        score+=0.5

print(score/i)

0.7421974219742198


In [ ]:
aweaimu a_file = open("data.pkl", "wb")
pickle.dump(vocab, a_file)
a_file.close()